In [ ]:
import numpy as np
from PIL import Image, ImageFilter

In [ ]:
print('is this alive')

In [ ]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    image = image.filter(ImageFilter.GaussianBlur(radius=2))  # Apply Gaussian blur for noise reduction
    return image


In [ ]:
def histogram_equalization(image):
    histogram, bins = np.histogram(np.array(image).flatten(), 256, density=True)
    cumulative_distribution = histogram.cumsum()
    cumulative_distribution = 255 * cumulative_distribution / cumulative_distribution[-1]

    equalized_image = np.interp(image, bins[:-1], cumulative_distribution)
    result = Image.fromarray(equalized_image.astype(np.uint8).reshape((image.size[1], image.size[0])))
    return result


In [ ]:
def sobel_filter(image):
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2], 
                        [-1, 0, 1]])    
    sobel_y = np.array([[1, 2, 1],
                        [0, 0, 0],
                        [-1, -2, -1]])

    gradient_x = image.filter(ImageFilter.Kernel((3, 3), sobel_x.flatten() ,1,0))
    gradient_y = image.filter(ImageFilter.Kernel((3, 3), sobel_y.flatten(),1,0))

    # gradient_x = np.abs(image.filter(ImageFilter.Kernel((3, 3), sobel_x)))

    gradient_magnitude = np.sqrt(np.square(gradient_x) + np.square(gradient_y))
    
    result = gradient_magnitude.reshape((image.size[1], image.size[0]))
    result = gradient_magnitude.astype(np.uint8)
    result = Image.fromarray(result)
    return result


In [ ]:
def laplace_filter(image):
    laplace = np.array([[-1, -1, -1],
                        [-1, 8, -1],
                        [-1, -1, -1]])

    gradient = image.filter(ImageFilter.Kernel((3, 3), laplace.flatten() ,1,0))

    return gradient

In [ ]:
def binarize_image(image, threshold):
    image = np.array(image)
    return Image.fromarray(((image > threshold)*255).astype(np.uint8))

In [ ]:
def find_circle(image):
    # hough Transform
    img_array=np.array(image)
    h, w = image.height, image.width
    min_radius = 2
    max_radius = min(w, h)/2

    circles = []
    accumulator = np.zeros((h, w, max_radius - min_radius + 1), dtype=int)

    for y in range(h):
        for x in range(w):
            if img_array[y, x] > 0:
                for r in range(min_radius, max_radius + 1):
                    for theta in range(0, 360):
                        a = int(x - r * np.cos(np.deg2rad(theta)))
                        b = int(y - r * np.sin(np.deg2rad(theta)))

                        if a >= 0 and a < w and b >= 0 and b < h:
                            accumulator[b, a, r - min_radius] += 1

    # Find the circles with sufficient votes
    for y in range(h):
        for x in range(w):
            for r in range(min_radius, max_radius + 1):
                if accumulator[y, x, r - min_radius] > 0.9 * accumulator.max():
                    circles.append((x, y, r))

    accumulator = np.zeros 
    # find coin circle
    # blackout surrounding background

    coin_regions = []

    for circle in circles:
        x, y, r = circle
        coin_region = image.crop((x - r, y - r, x + r, y + r))
        coin_regions.append(coin_region)
    Image.fromarray((coin_regions[0]).astype(np.uint8)).show
    return coin_regions

In [ ]:
def pattern_matching(image, pattern):
    
    difference = np.abs(np.array(image) - np.array(pattern))
    if np.mean(difference) < 10:  # should adjust
        return True

    return False


In [ ]:
def process(image_path):
    image = preprocess_image(image_path)
    image = histogram_equalization(image)
    image = laplace_filter(image)
    image = histogram_equalization(image)
    image = binarize_image(image, 205)
    circles = find_circle(image)
    
    return circles

In [ ]:
def detect_coins(image_path):
    circles = process(image_path)
    coin_pattern_10 = process("coin_template/10.jpeg")[0]
    coin_pattern_10_2 = process("coin_template/10_2.jpeg")[0]
    coin_pattern_50 = process("coin_template/50.jpg")[0]
    coin_pattern_50_2 = process("coin_template/50_2.png")[0]
    coin_pattern_100 = process("coin_template/100.jpg")[0]
    coin_pattern_100_2 = process("coin_template/100_2.png")[0]
    coin_pattern_500 = process("coin_template/500.png")[0]
    coin_pattern_500_2 = process("coin_template/500_2.png")[0]


    for circle in circles:
        if pattern_matching(circle, coin_pattern_10) or pattern_matching(circle, coin_pattern_10_2):
            print("10won Coin detected!")
            coin_count[0]+=1
        elif pattern_matching(circle, coin_pattern_50) or pattern_matching(circle, coin_pattern_50_2):
            print("50won Coin detected!")
            coin_count[1]+=1
        elif pattern_matching(circle, coin_pattern_100)or pattern_matching(circle, coin_pattern_100_2):
            print("100won Coin detected!")
            coin_count[2]+=1
        elif pattern_matching(circle, coin_pattern_500)or pattern_matching(circle, coin_pattern_500_2):
            print("500won Coin detected!")
            coin_count[3]+=1

    coin_count = [0 for _ in range(4)]

    # Load the pattern image of a coin to match against


    



Test

In [ ]:
image_path = ("coin_template/10.jpeg")

image = process(image_path)

image.show()

In [ ]:
detect_coins("coin_template/10.jpeg")